In [1]:
from cupy import ndarray
from typing import List, Any, Tuple, Set

import cupy
import nodeleys as ndl
import sys
import copy
import tensorflow as tf

def t_update(ndl):
  pending_for_deletion = []
  for i in sys.modules:
    if i.split('.')[0] == 'nodeleys':
      pending_for_deletion.append(i)

  # print(pending_for_deletion)
    
  for i in pending_for_deletion:
    del sys.modules[i]

  import nodeleys as ndl
  return ndl

2024-10-08 14:08:41.005343: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 14:08:41.015764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 14:08:41.030640: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 14:08:41.034251: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 14:08:41.047327: I tensorflow/core/platform/cpu_feature_guar

In [58]:
import time

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,516)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,516)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(516,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(516,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

gamma0 = node_matmul(data_x, w0, 'gamma0')
y0 = node_add(gamma0, b0, 'y0')

gamma1 = node_matmul(y0, w1, 'gamma1')
y1 = node_add(gamma1, b1, 'y1')

gamma1r = node_matmul(y0, w1r, 'gamma1r')
y1rs = node_add(gamma1r, b1r, 'y1rs')

y1r = Switch(y1rs, [gamma1r], [node_mul(y1rs, 0.), node_mul(RANX, y1rs), node_mul(y1rs, 0.)], ['x<=0', 'x>0', 'x<=0'], 'y1r').compile()
# y1r = Virtual([y1r_pre], [node_mul(y1r_pre, 0.), y1r_pre], ['[0]<0', '[0]>=0'], 'y1r').compile()

gamma_rR = node_matmul(y1r, rR, 'gamma_rR')
y_rR = node_add(gamma_rR, bR, 'y_rR')

gamma_rL = node_matmul(y1r, rL, 'gamma_rL')
y_rL = node_add(gamma_rL, bL, 'y_rL')

y_rRL = node_add(y_rR, y_rL, 'y_rRL')
y_rRL = node_div(5., node_add(5., y_rRL))

merge = node_add(y1r, y_rRL, 'merge')
merge2 = node_add(merge, y1, 'merge2')

gamma2 = node_matmul(merge2, w2, 'gamma2')
y2 = node_add(gamma2, b2, 'y2')

diff = node_sub(y2, data_y, 'diff')
squared = node_pow(diff, 2., 'squared')
redsum = node_redsum(squared, 0, 'redsum')
loss = node_div(redsum, 16., 'loss')

with tf.GradientTape(persistent=True) as g:
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  tf_gamma0 = tf_data_x @ tf_w0
  tf_y0 = tf_gamma0 + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  print(id(tf_gamma1r))
  print(id(tf_gamma1rs))
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  
start_time = time.time()
grads = g.gradient(tf_loss, [tf_w0, tf_w1, tf_w2, tf_b0, tf_gamma_rR])
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
loss.adic.set_as_objective()
tr = []
loss.adic.begin_backprop(tracing=False, traces=tr)
w0.get_gradient()
w1.get_gradient()
w2.get_gradient()
b0.get_gradient()
print("--- %s seconds ---" % (time.time() - start_time))

139675480337360
139675480341392
--- 0.006412029266357422 seconds ---
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[  -70.30361795   184.90510416   174.601976   ...    18.37730079
    -48.71095836   141.26876165]
 [-2603.35753478  -448.69451005   837.66135326 ...   607.91020821
    767.20913      323.37527932]
 [  153.3160691    145.82640578   -64.35829077 ...  -202.83654486
     53.59597474    57.58094397]
 ...
 [ -653.83530846  1684.32158775  4655.5048778  ...   599.55382566
   -269.77455727   342.54764375]
 [  377.04126812   200.37490756    27.33831238 ...  -128.32926958
     29.54842173  -260.11906441]
 [ -274.78516072  1225.40299049  -525.70792783 ...   297.41351683
   -743.41198548  -223.9332323 ]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0

In [49]:
w0.get_gradient()

array([[-2216719.5704072 ,   626394.19438614,  1036660.34502255, ...,
         -702777.92528456,   504742.17586609,   706562.05247141],
       [-2811134.53332952,  -723578.90306667,  1548738.58727583, ...,
         -833941.6211343 ,   761030.65744399,  -473151.31407247],
       [ 1776133.20816927, -1271125.69306865, -1844830.63223404, ...,
          943392.38596944,    99485.02254197, -1124991.95308106],
       ...,
       [ 2913166.71718057, -2126224.01789967, -3290952.24563971, ...,
         1861586.61439851,  -581835.17322673, -1341909.18287818],
       [-1446029.15781861,  -905766.94022788,  1232509.50423724, ...,
          -34431.38682223,   223714.09886089,    76916.19403916],
       [ -118842.22707994,  1001365.32026086,   220777.97495195, ...,
         -354913.68735439,  -455888.06209672,   841034.92337224]])

In [50]:
grads[0]

<tf.Tensor: shape=(1516, 516), dtype=float64, numpy=
array([[-2216719.57040721,   626394.19438614,  1036660.34502255, ...,
         -702777.92528457,   504742.17586609,   706562.05247141],
       [-2811134.53332953,  -723578.90306667,  1548738.58727583, ...,
         -833941.6211343 ,   761030.657444  ,  -473151.31407247],
       [ 1776133.20816927, -1271125.69306865, -1844830.63223405, ...,
          943392.38596944,    99485.02254197, -1124991.95308107],
       ...,
       [ 2913166.71718056, -2126224.01789968, -3290952.2456397 , ...,
         1861586.61439851,  -581835.17322673, -1341909.18287819],
       [-1446029.1578186 ,  -905766.94022788,  1232509.50423725, ...,
          -34431.38682223,   223714.09886089,    76916.19403916],
       [ -118842.22707994,  1001365.32026086,   220777.97495195, ...,
         -354913.68735439,  -455888.06209672,   841034.92337224]])>

In [69]:
ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
x = ndl.Node(cupy.random.normal(size=(10,10)))
print(x.tensor)
node_relu(x).tensor

[[-0.04710433  0.54420619  1.17574159  0.21813486  1.68257626  0.3986281
  -0.34513129 -0.58590319  0.29687417  1.85978098]
 [-1.28544328 -1.28013757 -0.01488046  0.05005914 -2.13714622  0.64959816
  -0.90078925 -1.66003304 -0.50783626 -0.17317154]
 [-0.63254889  0.43727574  0.17025356  0.87201782  0.36658144  1.82580419
  -1.26871636 -0.51437269  0.52708737  0.86097004]
 [-0.46030965 -0.62733715 -1.62119988 -1.27459283  1.01374223  1.08245333
   0.4510399   0.50277866 -0.75068139  0.22130257]
 [ 0.53317996  0.26758735 -1.95538583 -0.49527181 -0.04385985 -0.17831207
  -2.37670143 -1.80776075 -1.16241916  0.33524723]
 [-0.57546061  1.10798424 -2.06029833 -0.56538354  0.05843647  0.07421798
   2.23315375  1.61805131  0.06551809 -0.70496261]
 [ 1.06541693 -0.28347132  0.28232949  0.79915137  1.2663597   0.5415899
   0.87491735  1.6707235  -0.29585109 -0.90180896]
 [ 1.15023611 -0.97510714 -0.23937976 -0.00483921  1.0582596   0.37728426
  -0.15567885 -0.1979183   0.49094529  1.40132904]
 [

array([[0.        , 0.54420619, 1.17574159, 0.21813486, 1.68257626,
        0.3986281 , 0.        , 0.        , 0.29687417, 1.85978098],
       [0.        , 0.        , 0.        , 0.05005914, 0.        ,
        0.64959816, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.43727574, 0.17025356, 0.87201782, 0.36658144,
        1.82580419, 0.        , 0.        , 0.52708737, 0.86097004],
       [0.        , 0.        , 0.        , 0.        , 1.01374223,
        1.08245333, 0.4510399 , 0.50277866, 0.        , 0.22130257],
       [0.53317996, 0.26758735, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.33524723],
       [0.        , 1.10798424, 0.        , 0.        , 0.05843647,
        0.07421798, 2.23315375, 1.61805131, 0.06551809, 0.        ],
       [1.06541693, 0.        , 0.28232949, 0.79915137, 1.2663597 ,
        0.5415899 , 0.87491735, 1.6707235 , 0.        , 0.        ],
       [1.15023611, 0.        , 0.       

In [ ]:
img.tensor

In [ ]:
flatten.tensor

In [ ]:
w0.get_gradient()

In [ ]:
grads[0]

# Explore Lab

In [ ]:
tf.sqrt(784.)

In [ ]:
import cupy

a = cupy.array([[
  [1,2,3],
  [4,5,6],
  [7,8,9]
],[
  [7,8,9],
  [10,11,12],
  [0,1,2]
]])

cupy.max(a, axis=[-2,-1])

In [8]:
data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,784*2)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,784*2)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

with tf.GradientTape(persistent=True) as g:
  tf_img = tf.Variable(tf.random.normal(shape=(1516, 2, 28, 28), dtype=tf.float64))
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  tf_flatten = tf.reshape(tf_img, shape=(tf_img.shape[0], -1))
  # tf_gamma0 = tf_data_x @ tf_w0
  tf_y0 = tf_flatten + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  

In [ ]:
tf_flatten[0,900]

In [ ]:
tf_img[0,1,4,4]

In [ ]:
tf.reshape(g.gradient(tf_loss, tf_flatten), shape=tf_img.shape)

In [ ]:
g.gradient(tf_loss, tf_img)

In [ ]:
g.gradient(tf_loss, tf_img)[0,:,:,0]

# Explore Lab 2

In [ ]:
import time

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
img = ndl.Node(cupy.random.normal(size=(1516,2,28,28)), name='img')

data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,784*2)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,784*2)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

flatten = node_flatten(img, 'flatten')

gamma0 = node_matmul(data_x, w0, 'gamma0')
y0 = node_add(flatten, b0, 'y0')

gamma1 = node_matmul(y0, w1, 'gamma1')
y1 = node_add(gamma1, b1, 'y1')

gamma1r = node_matmul(y0, w1r, 'gamma1r')
y1rs = node_add(gamma1r, b1r, 'y1rs')

y1r = Switch(y1rs, [gamma1r], [node_mul(y1rs, 0.), node_mul(RANX, y1rs), node_mul(y1rs, 0.)], ['x<=0', 'x>0', 'x<=0'], 'y1r').compile()
# y1r = Virtual([y1r_pre], [node_mul(y1r_pre, 0.), y1r_pre], ['[0]<0', '[0]>=0'], 'y1r').compile()

gamma_rR = node_matmul(y1r, rR, 'gamma_rR')
y_rR = node_add(gamma_rR, bR, 'y_rR')

gamma_rL = node_matmul(y1r, rL, 'gamma_rL')
y_rL = node_add(gamma_rL, bL, 'y_rL')

y_rRL = node_add(y_rR, y_rL, 'y_rRL')
y_rRL = node_div(5., node_add(5., y_rRL))

merge = node_add(y1r, y_rRL, 'merge')
merge2 = node_add(merge, y1, 'merge2')

gamma2 = node_matmul(merge2, w2, 'gamma2')
y2 = node_add(gamma2, b2, 'y2')

diff = node_sub(y2, data_y, 'diff')
squared = node_pow(diff, 2., 'squared')
redsum = node_redsum(squared, 0, 'redsum')
loss = node_div(redsum, 16., 'loss')

with tf.GradientTape(persistent=True) as g:
  tf_img = tf.Variable(tf.convert_to_tensor(img.tensor.get()))
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  # tf_gamma0 = tf_data_x @ tf_w0
  tf_flatten = tf.reshape(tf_img, shape=(tf_img.shape[0], -1))
  tf_y0 = tf_flatten + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  print(id(tf_gamma1r))
  print(id(tf_gamma1rs))
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  
start_time = time.time()
grads = g.gradient(tf_loss, [tf_w0, tf_w1, tf_w2, tf_b0, tf_gamma_rR])
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
loss.adic.set_as_objective()
loss.adic.begin_backprop()
w0.get_gradient()
w1.get_gradient()
w2.get_gradient()
b0.get_gradient()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
g.gradient(tf_loss, tf_flatten)

In [ ]:
flatten.get_gradient()

# Exploration: Numpy for Convolution

In [1]:
import numpy as np

a = np.array([[0,  1,  2,  3,  4],
              [ 5,  6,  7,  8,  9],
              [10, 11, 12, 13, 14],
              [15, 16, 17, 18, 19],
              [20, 21, 22, 23, 24]], dtype=np.float64)

sub_shape = (3,3)
view_shape = tuple(np.subtract(a.shape, sub_shape) + 1) + sub_shape
strides = a.strides + a.strides

sub_matrices = np.lib.stride_tricks.as_strided(a,view_shape,strides)

In [5]:
a = np.random.uniform(size=(20,3,6,6), high=100).astype(int)
sub_shape = (20,3,3,3)
view_shape = tuple(np.subtract(a.shape, sub_shape) + 1) + sub_shape
strides = a.strides + a.strides
sub_matrices = np.lib.stride_tricks.as_strided(a, view_shape[2:], strides[2:])

In [ ]:
a.shape

In [ ]:
kernel_w = 3
stride_w = 1

w = 1 + int((a.shape[2] - kernel_w)/stride_w)


In [ ]:
view_shape

In [ ]:
a[0]

In [ ]:
sub_matrices.shape

In [ ]:
sub_matrices[1,0,0]

In [ ]:
view_shape

In [ ]:
strides

In [56]:


new_width = 1 + int((domain_width - kernel_width)/stride_width)
new_height = 1 + int((domain_height - kernel_height)/stride_height)

pick_shape = (new_height, new_width, batch_size, channel, kernel_height, kernel_width)

block_strides = cupy.array((block.strides + block.strides)[2:])
block_stride_strides = cupy.array((block.strides[2]*(stride_height-1), block.strides[3]*(stride_width-1), 0, 0,0,0))

memory_skip = (block_strides+block_stride_strides).get()

sub_block = cupy.lib.stride_tricks.as_strided(block, pick_shape, memory_skip)

In [ ]:
print(block[0])

In [ ]:
print(sub_block[1,1,0])

# Application

In [2]:
from cupy import ndarray
from typing import List, Any, Tuple, Set

import cupy
import nodeleys as ndl
import sys
import copy
import tensorflow as tf

def t_update(ndl):
  pending_for_deletion = []
  for i in sys.modules:
    if i.split('.')[0] == 'nodeleys':
      pending_for_deletion.append(i)

  # print(pending_for_deletion)
    
  for i in pending_for_deletion:
    del sys.modules[i]

  import nodeleys as ndl
  return ndl

In [13]:
batch_size = 20
channel = 3

kernel_width = 3
kernel_height = 3

domain_width = 28
domain_height = 28

stride_width = 2
stride_height = 2

block = cupy.random.uniform(size=(batch_size, channel, domain_width, domain_height), high=100).astype(int)
kernel = cupy.random.uniform(size=(5, channel, kernel_width, kernel_height), high = 10).astype(int)

In [17]:
ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

blocks = ndl.Node(block, name='blocks')
kernels = ndl.Node(kernel, name='kernels')

conv2d = node_conv2d(blocks, kernels, (1,1), 'conv2d')
maxpool2d = node_maxpool2d(conv2d, pool_size=(3,2), strides=(2,2), name='maxpool2d')
flatten = node_flatten(maxpool2d, 'flatten')

In [ ]:
import time
def tfct(node):
  return tf.convert_to_tensor(node.tensor.get(), dtype=tf.float64)

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

# dataset
img = ndl.Node(cupy.random.normal(scale=3.0, size=(166, 3, 28, 28)), name='img')
tf_img = tfct(img)

data_y = ndl.Node(cupy.random.normal(scale=10., size=(166, 1)), name='data_y')
tf_data_y = tfct(data_y)

# config
kernels = ndl.Node(cupy.random.normal(size=(2, 3, 3, 3)), name= 'kernel', is_trainable=True)
tf_kernels = tfct(kernels)
W1 = ndl.Node(cupy.random.normal(size=(1352, 500)), name='W1', is_trainable=True)
tf_W1 = tfct(W1)
W2 = ndl.Node(cupy.random.normal(size=(500, 32)), name='W2', is_trainable=True)
tf_W2 = tfct(W2)
W3 = ndl.Node(cupy.random.normal(size=(32, 1)), name='W3', is_trainable=True)
tf_W3 = tfct(W3)

# conv2d
conv2d = node_conv2d(img, kernels, (1,1), name='conv2d')

# flatten
flatten = node_flatten(conv2d, name='flatten')

# dense
dense1 = node_matmul(flatten, W1, name='dense1') # 1352 -> 500
dense2 = node_matmul(dense1, W2, name='dense2') # 500 -> 32
dense3 = node_matmul(dense2, W3, name='dense3') # 32 -> 1

delta = node_sub(dense3, data_y, name='delta')
L = node_redsum(delta, axis=0, name='redsum')
tf_L = tfct(L)

L.adic.set_as_objective()
start = time.time()
_, _ = L.adic.begin_backprop()
end = time.time()
print(end - start)

In [ ]:
with tf.GradientTape(persistent=True) as g:
  tf_kernels = tf.Variable(tf_kernels)
  tf_img = tf.Variable(tf_img)
  tf_conv2d = tf.nn.conv2d(tf_img, tf.transpose(tf_kernels, perm=[2,3,1,0]), strides=[1,1], padding='VALID', data_format='NCHW')
  tf_flatten = tf.keras.layers.Flatten(dtype=tf.float64)(tf_conv2d)
  tf_dense1 = tf_flatten @ tf.Variable(tf_W1)
  tf_dense2 = tf_dense1 @ tf.Variable(tf_W2)
  tf_dense3 = tf_dense2 @ tf.Variable(tf_W3)
  tf_delta = tf_dense3 - tf.Variable(tf_data_y)
  tf_L = tf.math.reduce_sum(input_tensor=tf_delta, axis=0, keepdims=True)

In [ ]:
# g.gradient(tf_L, tf_img)
img.get_gradient()

In [ ]:

g.gradient(tf_L, [tf_img])


In [ ]:
a = cupy.array([1,2,3]).get()
a

In [ ]:
import numpy as np

def consecutive_overlap_sum(arrays):
    # Stack all arrays into a 2D array
    stacked = np.vstack(arrays)
    
    # Create a triangular mask to handle shifts
    length = stacked.shape[1]
    tri_mask = np.tri(stacked.shape[0], length, dtype=int)
    
    # Element-wise multiply the mask with the stacked array
    shifted_stacked = stacked * tri_mask[:, :length]
    
    # Sum along the columns to get the final result
    result = shifted_stacked.sum(axis=0)
    
    return result

# Example with 100 arrays of length 4
arrays = [np.random.randint(1, 10, 4) for _ in range(100)]
print(arrays)
result = consecutive_overlap_sum(arrays)
print(result)


In [28]:
import cupy as np
import time

In [ ]:
N = 256
H = 128
W = 128
a = np.random.uniform(high=100, size=(N,28,H,W)).astype(int)

start = time.time()
result = np.zeros(shape=(N+3,28,H,W))
for i in range(N): 
  result[i+3,:,:,:] += a[i]
end = time.time()

print(end-start)

# print(a[0],'\n')
# print(a[1],'\n')
# print(a[2],'\n')
# print(a[3],'\n')

print(result.shape)

In [ ]:
import numpy as np

# Generate random 3x3 matrices
a = np.random.uniform(high=100, size=(3, 3)).astype(int)
b = np.random.uniform(high=100, size=(3, 3)).astype(int)
c = np.random.uniform(high=100, size=(3, 3)).astype(int)

def vectorized_consecutive_pad_sum(*matrices):
    # Number of matrices and their shape
    num_matrices = len(matrices)
    n, m = matrices[0].shape

    # Create a large matrix (3D) for all padded matrices
    max_padding = num_matrices - 1
    padded_matrices = np.zeros((num_matrices, n, m + max_padding))

    # Use slicing to place each matrix in the correct padded position
    matrix_array = np.array(matrices)
    row_index = np.arange(n)  # All rows
    col_index = np.arange(m)  # All columns

    # Broadcast each matrix into the larger array with the appropriate padding
    padded_matrices[:, row_index, col_index + np.arange(num_matrices)] = matrix_array

    # Sum along the first axis (over all matrices)
    result = np.sum(padded_matrices, axis=0)

    return result

# Call the function
result = vectorized_consecutive_pad_sum(a, b, c)

print("Matrix A:\n", a, "\n")
print("Matrix B:\n", b, "\n")
print("Matrix C:\n", c, "\n")
print("Resulting Sum:\n", result)


In [40]:
def tfct(node):
  return tf.convert_to_tensor(node.tensor.get(), dtype=tf.float64)

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch
from nodeleys.model import NodeleysModel

# dataset
img = ndl.Node(cupy.random.normal(size=(20, 3, 28, 28)), name='img')
data_y = ndl.Node(cupy.random.normal(size=(20, 1)), name='data_y')

class Model(NodeleysModel):
  def __init__(self):
    self.kernels = ndl.Node(cupy.random.normal(size=(2, 3, 3, 3)), name= 'kernel', is_trainable=True)
    self.W1 = ndl.Node(cupy.random.normal(size=(1352, 500)), name='W1', is_trainable=True)
    self.W2 = ndl.Node(cupy.random.normal(size=(500, 32)), name='W2', is_trainable=True)
    self.W3 = ndl.Node(cupy.random.normal(size=(32, 1)), name='W3', is_trainable=True)

  def graph(self):
    conv2d = node_conv2d(img, self.kernels, (1,1), name='conv2d')
    flatten = node_flatten(conv2d, name='flatten')

    dense1 = node_matmul(flatten, self.W1, name='dense1') # 1352 -> 500
    dense2 = node_matmul(dense1, self.W2, name='dense2') # 500 -> 32
    dense3 = node_matmul(dense2, self.W3, name='dense3') # 32 -> 1

    delta = node_sub(dense3, data_y, name='delta')
    L = node_redsum(delta, axis=0, name='redsum')
    tf_L = tfct(L)

    L.adic.set_as_objective()
    L.adic.begin_backprop()


In [41]:
x = Model()

In [ ]:
x.W1.get_gradient()

In [12]:
class Dense:
  def __init__(self, units):
    self.units = units
  
  def __call__(self, prev):
    

In [ ]:
def tfct(node):
  return tf.convert_to_tensor(node.tensor.get(), dtype=tf.float64)

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

# dataset
img = ndl.Node(cupy.random.normal(size=(20, 3, 28, 28)), name='img')
data_y = ndl.Node(cupy.random.normal(size=(20, 1)), name='data_y')

# config
kernels = ndl.Node(cupy.random.normal(size=(2, 3, 3, 3)), name= 'kernel', is_trainable=True)
W1 = ndl.Node(cupy.random.normal(size=(1352, 500)), name='W1', is_trainable=True)
W2 = ndl.Node(cupy.random.normal(size=(500, 32)), name='W2', is_trainable=True)
W3 = ndl.Node(cupy.random.normal(size=(32, 1)), name='W3', is_trainable=True)


# conv2d
conv2d = node_conv2d(img, kernels, (1,1), name='conv2d')

# flatten
flatten = node_flatten(conv2d, name='flatten')

# dense
dense1 = node_matmul(flatten, W1, name='dense1') # 1352 -> 500
dense2 = node_matmul(dense1, W2, name='dense2') # 500 -> 32
dense3 = node_matmul(dense2, W3, name='dense3') # 32 -> 1

delta = node_sub(dense3, data_y, name='delta')
L = node_redsum(delta, axis=0, name='redsum')

L.adic.set_as_objective()
L.adic.begin_backprop()


In [28]:
class G:
  def auxer(self, mex):
    def inner():
      print(type(mex))
      print('hex')
      mex()
    return inner
  
g = G()

@g.auxer
def aa():
  print('hast')
  pass

In [ ]:
aa()